In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/stsbenchmark')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/MyDrive/stsbenchmark'

**EMBEDDINGS SINGOLE FRASI POI CONCATENAZIONE, fasttext-wiki-news-subwords-300'***

In [ ]:
import csv
import pandas as pd
import json
import string
import re
import numpy as np
import plotly.express as px

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn import metrics

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

In [ ]:
columns=['genre', 'file', 'year', 'index', 'score', 'sentence1', 'sentence2']
df_train = pd.read_csv('/content/drive/MyDrive/stsbenchmark/sts-train.csv', sep='\t', usecols=range(7), header=None, quoting=csv.QUOTE_NONE, names=columns, encoding='UTF-8')
df_dev = pd.read_csv('/content/drive/MyDrive/stsbenchmark/sts-dev.csv', sep='\t', usecols=range(7), header=None, quoting=csv.QUOTE_NONE, names=columns, encoding='UTF-8')
df_test = pd.read_csv('/content/drive/MyDrive/stsbenchmark/sts-test.csv', sep='\t', usecols=range(7), header=None, quoting=csv.QUOTE_NONE, names=columns, encoding='UTF-8')

In [ ]:
def pre_processing(df: pd.DataFrame) -> pd.DataFrame:
    df['genre'] = df['genre'].replace('main-', '', regex=True)
    df['genre'] = df['genre'].replace('forum', 'forums')
    df['year'] = df['year'].replace(r'\D', '', regex=True)
    df['score'] = MinMaxScaler().fit_transform(df[['score']])
    return df

In [ ]:
df_train = pre_processing(df_train)
df_dev = pre_processing(df_dev)
df_test = pre_processing(df_test)

#df_train = df_train.drop([2372]).reset_index(drop=True)
#df_test = df_test.drop([632, 636,637,667,670,679,687,702,723,737,739,740,759,773,789,841,845]).reset_index(drop=True)

--descrizione dataset--

**Data Processing**

In [ ]:
import string
import re

regex = '[' + string.punctuation + ']'
print(regex)

[!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]


In [ ]:
df_train['vocab1'] = df_train['sentence1'].apply(lambda x: sorted(str(re.sub(regex, '', x)).split()))  
df_train['vocab2'] = df_train['sentence2'].apply(lambda x: sorted(str(re.sub(regex, '', x)).split()))
df_dev['vocab1'] = df_dev['sentence1'].apply(lambda x: sorted(str(re.sub(regex, '', x)).split())) 
df_dev['vocab2'] = df_dev['sentence2'].apply(lambda x: sorted(str(re.sub(regex, '', x)).split()))
df_test['vocab1'] = df_test['sentence1'].apply(lambda x: sorted(str(re.sub(regex, '', x)).split())) 
df_test['vocab2'] = df_test['sentence2'].apply(lambda x: sorted(str(re.sub(regex, '', x)).split()))



In [ ]:
'''global_vocab1 = df_train['vocab1'].sum()
print(global_vocab1[:20])

global_vocab2 = df_train['vocab2'].sum()
print(global_vocab2[:20])'''

"global_vocab1 = df_train['vocab1'].sum()\nprint(global_vocab1[:20])\n\nglobal_vocab2 = df_train['vocab2'].sum()\nprint(global_vocab2[:20])"

In [ ]:
'''import nltk
counts1 = nltk.Counter(global_vocab1)
print(counts1)

counts2 = nltk.Counter(global_vocab2)
print(counts2)'''

'import nltk\ncounts1 = nltk.Counter(global_vocab1)\nprint(counts1)\n\ncounts2 = nltk.Counter(global_vocab2)\nprint(counts2)'

In [ ]:
counts1.most_common(30)

counts2.most_common(30)

[('a', 1971),
 ('the', 1957),
 ('in', 1719),
 ('A', 1488),
 ('is', 1204),
 ('to', 1121),
 ('of', 990),
 ('and', 802),
 ('on', 782),
 ('man', 526),
 ('The', 441),
 ('with', 419),
 ('for', 399),
 ('at', 350),
 ('woman', 336),
 ('are', 261),
 ('said', 242),
 ('that', 234),
 ('dog', 225),
 ('was', 213),
 ('playing', 193),
 ('US', 191),
 ('from', 186),
 ('by', 183),
 ('as', 171),
 ('an', 163),
 ('Two', 160),
 ('white', 158),
 ('black', 147),
 ('it', 138)]

In [ ]:
'''nltk.download('stopwords')
from nltk.corpus import stopwords

# vocab 1 without stopwords
vocab1_nostop = [w for w in global_vocab1 if w not in stopwords.words('english')]
counts1_nostop = nltk.Counter(vocab1_nostop)

# vocab 2 without stopwords
vocab2_nostop = [w for w in global_vocab2 if w not in stopwords.words('english')]
counts2_nostop = nltk.Counter(vocab2_nostop)'''

"nltk.download('stopwords')\nfrom nltk.corpus import stopwords\n\n# vocab 1 without stopwords\nvocab1_nostop = [w for w in global_vocab1 if w not in stopwords.words('english')]\ncounts1_nostop = nltk.Counter(vocab1_nostop)\n\n# vocab 2 without stopwords\nvocab2_nostop = [w for w in global_vocab2 if w not in stopwords.words('english')]\ncounts2_nostop = nltk.Counter(vocab2_nostop)"

removing stopwords


In [ ]:
'''from nltk.corpus import stopwords
for i in range(0,len(df_train)):
  df_train['sentence1'][i] = re.sub(regex,' ',df_train['sentence1'][i])
  df_train['sentence2'][i] = re.sub(regex,' ',df_train['sentence2'][i])
  words1 = df_train['sentence1'][i].lower().split()
  words2 = df_train['sentence2'][i].lower().split()
  df_train['sentence1'][i] = ' '.join([w for w in words1 if w not in stopwords.words('english')])
  df_train['sentence2'][i] = ' '.join([w for w in words2 if w not in stopwords.words('english')])

for i in range(0,len(df_dev)):
  df_dev['sentence1'][i] = re.sub(regex,' ',df_dev['sentence1'][i])
  df_dev['sentence2'][i] = re.sub(regex,' ',df_dev['sentence2'][i])
  words1 = df_dev['sentence1'][i].lower().split()
  words2 = df_dev['sentence2'][i].lower().split()
  df_dev['sentence1'][i] = ' '.join([w for w in words1 if w not in stopwords.words('english')])
  df_dev['sentence2'][i] = ' '.join([w for w in words2 if w not in stopwords.words('english')])

for i in range(0,len(df_test)):
  df_test['sentence1'][i] = re.sub(regex,' ',df_test['sentence1'][i])
  df_test['sentence2'][i] = re.sub(regex,' ',df_test['sentence2'][i])
  words1 = df_test['sentence1'][i].lower().split()
  words2 = df_test['sentence2'][i].lower().split()
  df_test['sentence1'][i] = ' '.join([w for w in words1 if w not in stopwords.words('english')])
  df_test['sentence2'][i] = ' '.join([w for w in words2 if w not in stopwords.words('english')])'''
 


"from nltk.corpus import stopwords\nfor i in range(0,len(df_train)):\n  df_train['sentence1'][i] = re.sub(regex,' ',df_train['sentence1'][i])\n  df_train['sentence2'][i] = re.sub(regex,' ',df_train['sentence2'][i])\n  words1 = df_train['sentence1'][i].lower().split()\n  words2 = df_train['sentence2'][i].lower().split()\n  df_train['sentence1'][i] = ' '.join([w for w in words1 if w not in stopwords.words('english')])\n  df_train['sentence2'][i] = ' '.join([w for w in words2 if w not in stopwords.words('english')])\n\nfor i in range(0,len(df_dev)):\n  df_dev['sentence1'][i] = re.sub(regex,' ',df_dev['sentence1'][i])\n  df_dev['sentence2'][i] = re.sub(regex,' ',df_dev['sentence2'][i])\n  words1 = df_dev['sentence1'][i].lower().split()\n  words2 = df_dev['sentence2'][i].lower().split()\n  df_dev['sentence1'][i] = ' '.join([w for w in words1 if w not in stopwords.words('english')])\n  df_dev['sentence2'][i] = ' '.join([w for w in words2 if w not in stopwords.words('english')])\n\nfor i in r

In [ ]:
len(df_train)

5749

**Word Embeddings**

In [ ]:
#df_train.head()

corpus_train1= (df_train['sentence1'])
corpus_train2 = (df_train['sentence2'])
corpus_dev1 = (df_dev['sentence1'])
corpus_dev2 = (df_dev['sentence2'])
corpus_test1= (df_test['sentence1'])
corpus_test2= (df_test['sentence2'])
corpus_train1


0                                  A plane is taking off.
1                         A man is playing a large flute.
2           A man is spreading shreded cheese on a pizza.
3                            Three men are playing chess.
4                             A man is playing the cello.
                              ...                        
5744           Severe Gales As Storm Clodagh Hits Britain
5745    Dozens of Egyptians hostages taken by Libyan t...
5746                         President heading to Bahrain
5747           China, India vow to further bilateral ties
5748     Putin spokesman: Doping charges appear unfounded
Name: sentence1, Length: 5749, dtype: object

In [ ]:
import gensim.downloader as api

model_e = api.load('fasttext-wiki-news-subwords-300')

[==================================================] 100.0% 958.5/958.4MB downloaded


In [ ]:
def sentence_to_embedding(sentence: str) -> np.ndarray:
    tokenized_sentences: list[str] = re.sub('\W', ' ', sentence).lower().split()
    tokenized_sentences = [sentence for sentence in tokenized_sentences if len(sentence) > 1]
    embedding = np.zeros(model_e.vector_size)
    for word in tokenized_sentences:
        if word in model_e:
            embedding += model_e[word]
    return embedding / len(sentence)

In [ ]:
sentence_train1 = np.array(list(map(sentence_to_embedding, corpus_train1)))
sentence_train2 = np.array(list(map(sentence_to_embedding, corpus_train2)))
sentence_dev1 = np.array(list(map(sentence_to_embedding, corpus_dev1)))
sentence_dev2 = np.array(list(map(sentence_to_embedding, corpus_dev2)))
sentence_test1 = np.array(list(map(sentence_to_embedding, corpus_test1)))
sentence_test2 = np.array(list(map(sentence_to_embedding, corpus_test2)))


In [ ]:
print(len(sentence_train1))
print(len(sentence_train2))
for i in range(0, len(sentence_train1[:,1])):
  for j in range(0, len(sentence_train1[1,:])):
    if np.isnan(sentence_train1[i,j]):
      print(i,j)

5749
5749


In [ ]:
#print(df_train['sentence1'][2372])

#print(df_train['sentence2'][2372])

In [ ]:
X_train = np.concatenate((sentence_train1, sentence_train2), 1) 

import torch
X_train = torch.from_numpy(X_train)
X_train.size()
for i in range(0, len(X_train[:,1])):
  for j in range(0, len(X_train[1,:])):
    if torch.isnan(X_train[i,j]):
      print(i,j)


In [ ]:
x_dev= np.concatenate((sentence_dev1, sentence_dev2), 1) 
x_dev = torch.from_numpy(x_dev)
for i in range(0, len(x_dev[:,1])):
  for j in range(0, len(x_dev[1,:])):
    if torch.isnan(x_dev[i,j]):
      print(i,j)

X_train = torch.cat([X_train, x_dev])
X_train.size()


torch.Size([7249, 600])

In [ ]:
x_test=np.concatenate((sentence_test1, sentence_test2), 1) 
flag = bool
x_test = torch.from_numpy(x_test)
for i in range(0, len(x_test[:,1])):
  flag = False
  for j in range(0, len(x_test[1,:])):
    if torch.isnan(x_test[i,j]):
      flag = True;
  if flag:
    print(i)
    
x_test.size()

torch.Size([1379, 600])

In [ ]:
'''from sklearn.manifold import TSNE

tsne_train = TSNE(n_components=2, n_iter=500, n_jobs=-1, verbose=2)
tsne_embedding_train = tsne_train.fit_transform(X_train)
x, y = np.transpose(tsne_embedding_train)
#x1, y1, z1 = np.transpose(tsne_embedding_train[:len(df_train)])
#x2, y2, z2 = np.transpose(tsne_embedding_train[len(df_train):])

tsne_test = TSNE(n_components=2, n_iter=500, n_jobs=-1, verbose=2)
tsne_embedding_test = tsne_test.fit_transform(x_test)
xt, yt = np.transpose(tsne_embedding_test)
#x1_test, y1_test, z1_test = np.transpose(tsne_embedding_test[:len(df_test)])
#x2_test, y2_test, z2_test = np.transpose(tsne_embedding_test[len(df_test):])'''

'from sklearn.manifold import TSNE\n\ntsne_train = TSNE(n_components=2, n_iter=500, n_jobs=-1, verbose=2)\ntsne_embedding_train = tsne_train.fit_transform(X_train)\nx, y = np.transpose(tsne_embedding_train)\n#x1, y1, z1 = np.transpose(tsne_embedding_train[:len(df_train)])\n#x2, y2, z2 = np.transpose(tsne_embedding_train[len(df_train):])\n\ntsne_test = TSNE(n_components=2, n_iter=500, n_jobs=-1, verbose=2)\ntsne_embedding_test = tsne_test.fit_transform(x_test)\nxt, yt = np.transpose(tsne_embedding_test)\n#x1_test, y1_test, z1_test = np.transpose(tsne_embedding_test[:len(df_test)])\n#x2_test, y2_test, z2_test = np.transpose(tsne_embedding_test[len(df_test):])'

In [ ]:
'''X_train= pd.DataFrame(columns=['datax','datay'])
for i in range(0,7248):
  new_row={'datax': x[i], 'datay': y[i]}
  X_train.loc[len(X_train)]=new_row

x_test=pd.DataFrame(columns=['datax','datay'])
for i in range(0,len(df_test)):
  new_row={'datax': xt[i], 'datay': yt[i]}
  x_test.loc[len(x_test)]=new_row

print(X_train.head(15),x_test.head(15))'''

"X_train= pd.DataFrame(columns=['datax','datay'])\nfor i in range(0,7248):\n  new_row={'datax': x[i], 'datay': y[i]}\n  X_train.loc[len(X_train)]=new_row\n\nx_test=pd.DataFrame(columns=['datax','datay'])\nfor i in range(0,len(df_test)):\n  new_row={'datax': xt[i], 'datay': yt[i]}\n  x_test.loc[len(x_test)]=new_row\n\nprint(X_train.head(15),x_test.head(15))"

In [ ]:
y_train = pd.concat([df_train['score'],df_dev['score']]).reset_index(drop=True)

In [ ]:
y_dev = df_dev['score']

In [ ]:
y_test = df_test['score']

len(y_train)

7249

**Training Data**

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
#print("MAE train: ", metrics.mean_absolute_error(y_train, model.predict(x_test))) 
print("MSE train: ",metrics.mean_squared_error(y_train, model.predict(X_train)))
print("RMSE train: ",np.sqrt(metrics.mean_squared_error(y_train, model.predict(X_train))))
print("r2: ",metrics.r2_score(y_train, model.predict(X_train)))

#print("MAE test: ", metrics.mean_absolute_error(y_test, model.predict(x_test))) 
print("MSE test: ",metrics.mean_squared_error(y_test, model.predict(x_test)))
print("RMSE test: ",np.sqrt(metrics.mean_squared_error(y_test, model.predict(x_test))))
print("r2: ", metrics.r2_score(y_test, model.predict(x_test)))

MSE train:  0.07121485442257883
RMSE train:  0.26686111448200694
r2:  0.1850516824464813
MSE test:  0.09750906639361108
RMSE test:  0.3122644174311429
r2:  -0.04825211793526818


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])
x_ax=np.array(range(0,len(y_test)))
# Add traces
fig.add_trace(
    go.Scatter(x=x_ax, y=y_test, name="True"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=x_ax, y=y_pred, name="Predicition"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Results"
)

# Set x-axis title
fig.update_xaxes(title_text="Couple of sentences")

# Set y-axes titles
fig.update_yaxes(title_text="True", secondary_y=True)
fig.update_yaxes(title_text="Prediction", secondary_y=True)

fig.show()

In [ ]:
from sklearn.linear_model import PoissonRegressor

model = PoissonRegressor()
model.fit(X_train, y_train)

PoissonRegressor()

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
print("MAE train: ", metrics.mean_absolute_error(y_train, model.predict(X_train))) 
print("MSE train: ",metrics.mean_squared_error(y_train, model.predict(X_train)))
print("RMSE train: ",np.sqrt(metrics.mean_squared_error(y_train, model.predict(X_train))))
print("r2: ",metrics.r2_score(y_train, model.predict(X_train)))

print("MAE test: ", metrics.mean_absolute_error(y_test, model.predict(x_test))) 
print("MSE test: ",metrics.mean_squared_error(y_test, model.predict(x_test)))
print("RMSE test: ",np.sqrt(metrics.mean_squared_error(y_test, model.predict(x_test))))
print("r2: ", metrics.r2_score(y_test, model.predict(x_test)))

MAE train:  0.253161876403856
MSE train:  0.08738261094318335
RMSE train:  0.2956054988378656
r2:  3.5703941522746696e-05
MAE test:  0.2614151930678656
MSE test:  0.09303741668601871
RMSE test:  0.30502035454378895
r2:  -0.0001805236721583725


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
x_ax=np.array(range(0,len(y_test)))
fig.add_trace(
    go.Scatter(x=x_ax, y=y_test, name="True"),
    secondary_y=True,)
fig.add_trace(
    go.Scatter(x=x_ax, y=y_pred, name="Predicition"),
    secondary_y=True,)
fig.update_layout(
    title_text="Results")
fig.update_xaxes(title_text="Couple of sentences")
fig.update_yaxes(title_text="True", secondary_y=True)
fig.update_yaxes(title_text="Prediction", secondary_y=True)

fig.show()

In [ ]:
from sklearn.svm import SVR
model = SVR()
parameters = {'C': [10],
             'epsilon': [0.1, 0.01],
             'gamma':['auto'],
             'kernel': ['linear', 'poly','rbf'],
             'degree': [2,3,5]
             }

#DEFINE YOUR GRIDSEARCH 
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(model, parameters, cv=3, verbose = 0)
#gs_dev = GridSearchCV(model, parameters, cv=3, verbose = 0)

gs = gs.fit(X_train,y_train)
#gs_dev = gs_dev.fit(x_dev,y_dev)

In [ ]:
print('**GRIDSEARCH RESULTS**')
print("Best score: %f using %s" % (gs.best_score_, gs.best_params_))
means = gs.cv_results_['mean_test_score']
stds = gs.cv_results_['std_test_score']
params = gs.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

'''print("Best score: %f using %s" % (gs_dev.best_score_, gs_dev.best_params_))
means = gs_dev.cv_results_['mean_test_score']
stds = gs_dev.cv_results_['std_test_score']
params = gs_dev.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

#Returns the coefficient of determination R^2 of the prediction.
#Explained variance score: 1 is perfect prediction'''


**GRIDSEARCH RESULTS**
Best score: -0.036859 using {'C': 10, 'degree': 2, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'linear'}
-0.036859 (0.029631) with: {'C': 10, 'degree': 2, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'linear'}
-0.102612 (0.029844) with: {'C': 10, 'degree': 2, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'poly'}
-0.078234 (0.022847) with: {'C': 10, 'degree': 2, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}
-0.050879 (0.039614) with: {'C': 10, 'degree': 2, 'epsilon': 0.01, 'gamma': 'auto', 'kernel': 'linear'}
-0.121453 (0.022408) with: {'C': 10, 'degree': 2, 'epsilon': 0.01, 'gamma': 'auto', 'kernel': 'poly'}
-0.094298 (0.023885) with: {'C': 10, 'degree': 2, 'epsilon': 0.01, 'gamma': 'auto', 'kernel': 'rbf'}
-0.036859 (0.029631) with: {'C': 10, 'degree': 3, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'linear'}
-0.102615 (0.029846) with: {'C': 10, 'degree': 3, 'epsilon': 0.1, 'gamma': 'auto', 'kernel': 'poly'}
-0.078234 (0.022847) with: {'C': 10, 'degree': 3, 'epsilon

'print("Best score: %f using %s" % (gs_dev.best_score_, gs_dev.best_params_))\nmeans = gs_dev.cv_results_[\'mean_test_score\']\nstds = gs_dev.cv_results_[\'std_test_score\']\nparams = gs_dev.cv_results_[\'params\']\nfor mean, stdev, param in zip(means, stds, params):\n    print("%f (%f) with: %r" % (mean, stdev, param))\n\n#Returns the coefficient of determination R^2 of the prediction.\n#Explained variance score: 1 is perfect prediction'

In [ ]:
model = SVR(C= 10, degree= 2, epsilon= 0.1, gamma= 'auto', kernel= 'linear')
model.fit(X_train, y_train)

SVR(C=10, degree=2, gamma='auto', kernel='linear')

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
#print("MAE train: ", metrics.mean_absolute_error(y_train, model.predict(X_train))) 
print("MSE train: ",metrics.mean_squared_error(y_train, model.predict(X_train)))
print("RMSE train: ",np.sqrt(metrics.mean_squared_error(y_train, model.predict(X_train))))
print("r2: ",metrics.r2_score(y_train, model.predict(X_train)))

#print("MAE test: ", metrics.mean_absolute_error(y_test, model.predict(x_test))) 
print("MSE test: ",metrics.mean_squared_error(y_test, model.predict(x_test)))
print("RMSE test: ",np.sqrt(metrics.mean_squared_error(y_test, model.predict(x_test))))
print("r2: ", metrics.r2_score(y_test, model.predict(x_test)))

MSE train:  0.07541458040444249
RMSE train:  0.27461715242213564
r2:  0.1369920514768438
MSE test:  0.08870834663183728
RMSE test:  0.29783946453053745
r2:  0.04635829595583019


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
x_ax=np.array(range(0,len(y_test)))
fig.add_trace(
    go.Scatter(x=x_ax, y=y_test, name="True"),
    secondary_y=True,)
fig.add_trace(
    go.Scatter(x=x_ax, y=y_pred, name="Predicition"),
    secondary_y=True,)
fig.update_layout(
    title_text="Results")
fig.update_xaxes(title_text="Couple of sentences")
fig.update_yaxes(title_text="True", secondary_y=True)
fig.update_yaxes(title_text="Prediction", secondary_y=True)

fig.show()

In [ ]:
from sklearn import linear_model
from sklearn.linear_model import BayesianRidge
model = linear_model.BayesianRidge()
model.fit(X_train, y_train)

BayesianRidge()

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
print("MAE train: ", metrics.mean_absolute_error(y_train, model.predict(X_train))) 
print("MSE train: ",metrics.mean_squared_error(y_train, model.predict(X_train)))
print("RMSE train: ",np.sqrt(metrics.mean_squared_error(y_train, model.predict(X_train))))
print("r2: ",metrics.r2_score(y_train, model.predict(X_train)))

print("MAE test: ", metrics.mean_absolute_error(y_test, model.predict(x_test))) 
print("MSE test: ",metrics.mean_squared_error(y_test, model.predict(x_test)))
print("RMSE test: ",np.sqrt(metrics.mean_squared_error(y_test, model.predict(x_test))))
print("r2: ", metrics.r2_score(y_test, model.predict(x_test)))

MAE train:  0.23105009598054238
MSE train:  0.07580225836956873
RMSE train:  0.2753220993120035
r2:  0.13255565252617718
MAE test:  0.24574733519114983
MSE test:  0.08684249480780985
RMSE test:  0.29469050681657505
r2:  0.06641677050213457


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
x_ax=np.array(range(0,len(y_test)))
fig.add_trace(
    go.Scatter(x=x_ax, y=y_test, name="True"),
    secondary_y=True,)
fig.add_trace(
    go.Scatter(x=x_ax, y=y_pred, name="Predicition"),
    secondary_y=True,)
fig.update_layout(
    title_text="Results")
fig.update_xaxes(title_text="Couple of sentences")
fig.update_yaxes(title_text="True", secondary_y=True)
fig.update_yaxes(title_text="Prediction", secondary_y=True)

fig.show()